In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

We're ready to build our first neural network. We will have multiple features we feed into our model, each of which will go through a set of perceptron models to arrive at a response which will be trained to our output.

Like many models we've covered, this can be used as both a regression or classification model.

First, we need to load our dataset. For this example we'll use The Museum of Modern Art in New York's [public dataset](https://media.githubusercontent.com/media/MuseumofModernArt/collection/master/Artworks.csv) on their collection.

In [49]:
artworks = pd.read_csv('https://media.githubusercontent.com/media/MuseumofModernArt/collection/master/Artworks.csv')

In [50]:
artworks.columns

Index(['Title', 'Artist', 'ConstituentID', 'ArtistBio', 'Nationality',
       'BeginDate', 'EndDate', 'Gender', 'Date', 'Medium', 'Dimensions',
       'CreditLine', 'AccessionNumber', 'Classification', 'Department',
       'DateAcquired', 'Cataloged', 'ObjectID', 'URL', 'ThumbnailURL',
       'Circumference (cm)', 'Depth (cm)', 'Diameter (cm)', 'Height (cm)',
       'Length (cm)', 'Weight (kg)', 'Width (cm)', 'Seat Height (cm)',
       'Duration (sec.)'],
      dtype='object')

We'll also do a bit of data processing and cleaning, selecting columns of interest and converting URL's to booleans indicating whether they are present.

In [51]:
# Select Columns.
artworks = artworks[['Artist', 'Nationality', 'Gender', 'Date', 'Department',
                    'DateAcquired', 'URL', 'ThumbnailURL', 'Height (cm)', 'Width (cm)']]

# Convert URL's to booleans.
artworks['URL'] = artworks['URL'].notnull()
artworks['ThumbnailURL'] = artworks['ThumbnailURL'].notnull()

# Drop films and some other tricky rows.
artworks = artworks[artworks['Department']!='Film']
artworks = artworks[artworks['Department']!='Media and Performance Art']
artworks = artworks[artworks['Department']!='Fluxus Collection']

# Drop missing data.
artworks = artworks.dropna()

In [52]:
artworks.head()

,Artist,Nationality,Gender,Date,Department,DateAcquired,URL,ThumbnailURL,Height (cm),Width (cm)
0,Otto Wagner,(Austrian),(Male),1896,Architecture & Design,1996-04-09,True,True,48.6000,168.9000
1,Christian de Portzamparc,(French),(Male),1987,Architecture & Design,1995-01-17,True,True,40.6401,29.8451
2,Emil Hoppe,(Austrian),(Male),1903,Architecture & Design,1997-01-15,True,True,34.3000,31.8000
3,Bernard Tschumi,(),(Male),1980,Architecture & Design,1995-01-17,True,True,50.8000,50.8000
4,Emil Hoppe,(Austrian),(Male),1903,Architecture & Design,1997-01-15,True,True,38.4000,19.1000


## Building a Model

Now, let's see if we can use multi-layer perceptron modeling (or "MLP") to see if we can classify the department a piece should go into using everything but the department name.

Before we import MLP from SKLearn and establish the model we first have to ensure correct typing for our data and do some other cleaning.

In [53]:
# Get data types.
artworks.dtypes

Artist           object
Nationality      object
Gender           object
Date             object
Department       object
DateAcquired     object
URL                bool
ThumbnailURL       bool
Height (cm)     float64
Width (cm)      float64
dtype: object

The `DateAcquired` column is an object. Let's transform that to a datetime object and add a feature for just the year the artwork was acquired.

In [54]:
artworks['DateAcquired'] = pd.to_datetime(artworks.DateAcquired)
artworks['YearAcquired'] = artworks.DateAcquired.dt.year
artworks['YearAcquired'].dtype

dtype('int64')

Great. Let's do some more miscellaneous cleaning.

In [55]:
# Remove multiple nationalities, genders, and artists.
artworks.loc[artworks['Gender'].str.contains('\) \('), 'Gender'] = '\(multiple_persons\)'
artworks.loc[artworks['Nationality'].str.contains('\) \('), 'Nationality'] = '\(multiple_nationalities\)'
artworks.loc[artworks['Artist'].str.contains(','), 'Artist'] = 'Multiple_Artists'

# Convert dates to start date, cutting down number of distinct examples.
artworks['Date'] = pd.Series(artworks.Date.str.extract(
    '([0-9]{4})', expand=False))[:-1]

# Final column drops and NA drop.
X = artworks.drop(['Department', 'DateAcquired', 'Artist', 'Nationality', 'Date'], 1)

In [56]:
X.head()

,Gender,URL,ThumbnailURL,Height (cm),Width (cm),YearAcquired
0,(Male),True,True,48.6000,168.9000,1996
1,(Male),True,True,40.6401,29.8451,1995
2,(Male),True,True,34.3000,31.8000,1997
3,(Male),True,True,50.8000,50.8000,1995
4,(Male),True,True,38.4000,19.1000,1997


In [57]:
# Create dummies separately.
artists = pd.get_dummies(artworks.Artist)
nationalities = pd.get_dummies(artworks.Nationality)
dates = pd.get_dummies(artworks.Date)

In [58]:
# Concat artists with other variables
X = pd.get_dummies(X, sparse=True)
X = pd.concat([X, nationalities, dates], axis=1)

Y = artworks.Department

In [59]:
# Alright! We've done our prep, let's build the model.
# Neural networks are hugely computationally intensive.
# This may take several minutes to run.

# Import the model.
from sklearn.neural_network import MLPClassifier

# Establish and fit the model, with a single, 1000 perceptron layer.
mlp = MLPClassifier(hidden_layer_sizes=(1000,))
mlp.fit(X, Y)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(1000,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [60]:
mlp.score(X, Y)

0.63597274527312964

In [61]:
Y.value_counts()/len(Y)

Prints & Illustrated Books    0.523363
Photography                   0.225356
Architecture & Design         0.112416
Drawings                      0.104137
Painting & Sculpture          0.034728
Name: Department, dtype: float64

In [62]:
from sklearn.model_selection import cross_val_score
cross_val_score(mlp, X, Y, cv=5)

c:\users\ryan\appdata\local\programs\python\python36\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:565: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")
c:\users\ryan\appdata\local\programs\python\python36\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:565: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


array([ 0.51746894,  0.40937545,  0.48874114,  0.53897301,  0.49817988])

Now we got a lot of information from all of this. Firstly we can see that the model seems to overfit, though there is still so remaining performance when validated with cross validation. This is a feature of neural networks that aren't given enough data for the number of features present. _Neural networks, in general, like_ a lot _of data_. You may also have noticed something also about neural networks: _they can take a_ long _time to run_. Try increasing the layer size by adding a zero. Feel free to interrupt the kernel if you don't have time...

Also note that we created bools for artist's name but left them out. Both of the above points are the reason for that. It would take much longer to run and it would be much more prone to overfitting.

## Model parameters

Now, before we move on and let you loose with some tasks to work on the model, let's go over the parameters.

We included one parameter: hidden layer size. Remember in the previous lesson, when we talked about layers in a neural network. This tells us how many and how big to make our layers. Pass in a tuple that specifies each layer's size. Our network is 1000 neurons wide and one layer. (100, 4, ) would create a network with two layers, one 100 wide and the other 4.

How many layers to include is determined by two things: computational resources and cross validation searching for convergence. It's generally less than the number of input variables you have.

You can also set an alpha. Neural networks like this use a regularization paramater that penalizes large coefficients just like we discussed in the advanced regression section. Alpha scales that penalty.

Lastly, we'll discuss the activation function. The activation function determines whether the output from an individual perceptron is binary or continuous. By default this is a binary function ('relu', or 'rectified linear unit function'). In the exercise we went through earlier we used this binary function, but we discussed the _sigmoid_ as a reasonable alternative. The _sigmoid_ (called 'logistic' by SKLearn because it's a 'logistic sigmoid function') allows for continuous variables between 0 and 1, which allows for a more nuanced model. It does come at the cost of increased computational complexity.

If you want to learn more about these, study [activation functions](https://en.wikipedia.org/wiki/Activation_function) and [multilayer perceptrons](https://en.wikipedia.org/wiki/Multilayer_perceptron). The [Deep Learning](http://www.deeplearningbook.org/) book referenced earlier goes into great detail on the linear algebra involved.

You could also just test the models with cross validation. Unless neural networks are your specialty cross validation should be sufficient.

For the other parameters and their defaults, check out the [MLPClassifier documentaiton](http://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html#sklearn.neural_network.MLPClassifier).

## Drill: Playing with layers

Now it's your turn. Using the space below, experiment with different hidden layer structures. You can try this on a subset of the data to improve runtime. See how things vary. See what seems to matter the most. Feel free to manipulate other parameters as well. It may also be beneficial to do some real feature selection work...

In [68]:
# Your code here. Experiment with hidden layers to build your own model.

#Use a 20% sample to train other models
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.8)

In [69]:
X_train.head()

,URL,ThumbnailURL,Height (cm),Width (cm),YearAcquired,Gender_(),Gender_(Female),Gender_(Male),Gender_(male),Gender_\(multiple_persons\),...,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
65667,True,True,23.800000,19.700000,1942,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
109201,True,True,17.600000,13.900000,1999,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
25243,False,False,17.900000,13.900000,1964,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
121709,False,False,21.748793,28.416307,2014,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
127972,True,False,24.765050,21.590043,2015,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [70]:
mlp = MLPClassifier(hidden_layer_sizes=(100,5))
mlp.fit(X_train, y_train)
print('Score:', mlp.score(X_train, y_train))
print('Cross Validation Scores:', cross_val_score(mlp, X_train, y_train, cv=5))

Score: 0.523416646445
Cross Validation Scores: [ 0.52328967  0.57423581  0.52365931  0.54477069  0.52342802]


In [71]:
mlp = MLPClassifier(hidden_layer_sizes=(10,10))
mlp.fit(X_train, y_train)
print('Score:', mlp.score(X_train, y_train))
print('Cross Validation Scores:', cross_val_score(mlp, X_train, y_train, cv=5))

Score: 0.541033729677
Cross Validation Scores: [ 0.61669093  0.59024745  0.5603009   0.59864111  0.59286235]


In [77]:
mlp = MLPClassifier(hidden_layer_sizes=(10,10,10))
mlp.fit(X_train, y_train)
print('Score:', mlp.score(X_train, y_train))
print('Cross Validation Scores:', cross_val_score(mlp, X_train, y_train, cv=5))

Score: 0.597330744965
Cross Validation Scores: [ 0.6089277   0.63124697  0.6010677   0.55714632  0.60500121]


In [74]:
mlp = MLPClassifier(hidden_layer_sizes=(100,100))
mlp.fit(X_train, y_train)
print('Score:', mlp.score(X_train, y_train))
print('Cross Validation Scores:', cross_val_score(mlp, X_train, y_train, cv=5))

Score: 0.564183450619
Cross Validation Scores: [ 0.20257157  0.23532266  0.24508614  0.52123271  0.54552076]


In [75]:
#Feedback from Kevin: 

#Try perceptron model w/ 310 as first layer (this is how many features we have), 
#Then try adding layers beyond that (300, 10, 1), etc, to see how performance changes

mlp = MLPClassifier(hidden_layer_sizes=(300,))
mlp.fit(X_train, y_train)
print('Score:', mlp.score(X_train, y_train))
print('Cross Validation Scores:', cross_val_score(mlp, X_train, y_train, cv=5))

Score: 0.39825285125
Cross Validation Scores: [ 0.57908782  0.49417758  0.52972579  0.56078622  0.60257344]


In [76]:
mlp = MLPClassifier(hidden_layer_sizes=(300,10,1))
mlp.fit(X_train, y_train)
print('Score:', mlp.score(X_train, y_train))
print('Cross Validation Scores:', cross_val_score(mlp, X_train, y_train, cv=5))

Score: 0.523368114535
Cross Validation Scores: [ 0.52328967  0.52328967  0.52341665  0.52341665  0.52342802]


In [78]:
mlp = MLPClassifier(hidden_layer_sizes=(300,10,10,10))
mlp.fit(X_train, y_train)
print('Score:', mlp.score(X_train, y_train))
print('Cross Validation Scores:', cross_val_score(mlp, X_train, y_train, cv=5))

Score: 0.523368114535
Cross Validation Scores: [ 0.62809316  0.63852499  0.46202378  0.59378792  0.5654285 ]


In [79]:
mlp = MLPClassifier(hidden_layer_sizes=(5,5,5,5,5))
mlp.fit(X_train, y_train)
print('Score:', mlp.score(X_train, y_train))
print('Cross Validation Scores:', cross_val_score(mlp, X_train, y_train, cv=5))

Score: 0.548604707595
Cross Validation Scores: [ 0.57544881  0.55288695  0.54938122  0.55569037  0.51444525]


In [80]:
###Three 10-perceptron layers seemed to yield the best and most stable results. Try changing the activation function to logistic.

mlp = MLPClassifier(hidden_layer_sizes=(10,10,10), activation = 'logistic')
mlp.fit(X_train, y_train)
print('Score:', mlp.score(X_train, y_train))
print('Cross Validation Scores:', cross_val_score(mlp, X_train, y_train, cv=5))

Score: 0.544333899539
Cross Validation Scores: [ 0.63415818  0.5725376   0.54040281  0.55471973  0.5472202 ]


In [81]:
#Adding the logistic activation function did not improve the model performance.
#Try going back to default ('relu' = rectified linear unit) and change alpha (the L2 regularization parameter)

mlp = MLPClassifier(hidden_layer_sizes=(10,10,10), alpha = 0.01)
mlp.fit(X_train, y_train)
print('Score:', mlp.score(X_train, y_train))
print('Cross Validation Scores:', cross_val_score(mlp, X_train, y_train, cv=5))

Score: 0.606551807814
Cross Validation Scores: [ 0.59801067  0.63925279  0.53069643  0.55399175  0.60378733]


In [82]:
#How to best approach feature selection? Easiest way is via regularization parameter alpha? What about PCA? Doesn't make 
#much sense to use PCA in this instance, as most of the data is boolean (due to dummy vars) and we have so many dimensions

#Another method would be to make a correlation matrix for the input variables (dropping highly correlated variables, or 
#combining them into one variable), but again due to the feature set we are dealing
#with, there will be virtually no correlation since we have so many boolean variables that are independent and mutually exclusive

#Of the non-boolean variables (like height and width), we likely don't want to drop those even if they are correlated because
#they probably also provide a significant amount of information for our model
